In [121]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Bernoulli, Normal, Categorical, Dirichlet, Uniform, Multinomial, Empirical
from edward.util import Progbar

Create some sample data:

In [2]:
sample_size = 1000
obs_cond = np.random.choice([0,1,2],[sample_size],p=[.7,.1,.2]) # cold = 0, flu = 1, allergies = 2

runny_nose_cold = np.random.choice([0,1],[sample_size],p=[.1,.9])
headache_cold = np.random.choice([0,1],[sample_size],p=[.5,.5])
fever_cold = np.random.choice([0,1],[sample_size],p=[.99,.01])

runny_nose_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])
headache_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])
fever_flu = np.random.choice([0,1],[sample_size],p=[.5,.5])

runny_nose_al = np.random.choice([0,1],[sample_size],p=[.1,.9])
headache_al = np.random.choice([0,1],[sample_size],p=[.99,.01])
fever_al = np.random.choice([0,1],[sample_size],p=[.99,.01])

runny_nose = np.stack([runny_nose_cold,runny_nose_flu,runny_nose_al])
headache = np.stack([headache_cold,headache_flu,headache_al])
fever = np.stack([fever_cold,fever_flu,fever_al])

In [3]:
obs_runny_nose = np.array([runny_nose[j,i] for i,j in enumerate(obs_cond)])
obs_headache = np.array([headache[j,i] for i,j in enumerate(obs_cond)])
obs_fever = np.array([fever[j,i] for i,j in enumerate(obs_cond)])

In [266]:
### prior_conds = Dirichlet(name='prior_probs',alpha=np.array([1.,1.,1.]))
cat_p = tf.Variable(tf.constant([.3,.3,.4]))
cond = Categorical(name='cond', p=tf.ones([sample_size,3])*cat_p)
    
W = tf.Variable(tf.zeros([3,3]),name='W') #Uniform(name='W',a=0.,b=1.,shape=[3,3])
    
runny_nose = Bernoulli(name='runny_nose',logits=tf.gather(W[0,:],cond))
headache = Bernoulli(name='headache',logits=tf.gather(W[1,:],cond))
fever = Bernoulli(name='fever',logits=tf.gather(W[2,:],cond))

In [267]:
inference = ed.MAP({},{cond:obs_cond,runny_nose:obs_runny_nose,headache:obs_headache,fever:obs_fever})

In [268]:
inference.run(n_iter=1000)

1000/1000 [100%] ██████████████████████████████ Elapsed: 2s | Loss: 1844.628


In [269]:
cat_p.eval()

array([ 1.40192449,  0.22793896,  0.38729435], dtype=float32)

In [14]:
tf.nn.sigmoid(W).eval()

array([[ 0.90935266,  0.48672545,  0.91666663],
       [ 0.49928072,  0.46902663,  0.0010181 ],
       [ 0.0086338 ,  0.47787613,  0.01041685]], dtype=float32)